In [1]:
#%load_ext autoreload
#%autoreload 2

In [2]:
import sys
sys.path.insert(0, '../src/20190927')

In [3]:
import util

/home/alexandre_maia/miniconda3/envs/rsna/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/alexandre_maia/miniconda3/envs/rsna/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/alexandre_maia/miniconda3/envs/rsna/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/alexandre_maia/mini

In [4]:
config = {
    'train_path': '../data/stage_1_train_images_jpg/',
    'test_path': '../data/stage_1_test_images_jpg/'
}

#test_df = util.read_testset()
df = util.read_trainset()

# train set (90%) and validation set (10%)
from sklearn.model_selection import ShuffleSplit
ss = ShuffleSplit(n_splits=5, test_size=0.1, random_state=42).split(df.index)

# will just do one fold
train_idx, valid_idx = next(ss)

print(len(train_idx), len(valid_idx))

606831 67426


In [5]:
gen = util.BalancedTrainDataGenerator(df.iloc[train_idx], 64, (224, 224))

building balanced train generator:  606831 ../data/stage_1_train_images_jpg/


In [6]:
from tensorflow.keras.applications.resnet50 import ResNet50

model = util.Model1(engine=ResNet50, input_dims=(224, 224), 
               batch_size=64, learning_rate=1e-3, 
               decay_rate=0.75, decay_steps=1, weights="imagenet", 
               verbose=1)

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


/home/alexandre_maia/miniconda3/envs/rsna/lib/python3.7/site-packages/keras_applications/resnet50.py:265: UserWarning: The output shape of `ResNet50(include_top=False)` has been changed since Keras 2.2.0.
  warnings.warn('The output shape of `ResNet50(include_top=False)` '


Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


In [8]:
from math import floor
import tensorflow as tf

callbacks = [
    tf.keras.callbacks.ModelCheckpoint('../data/weights.{epoch:02d}.hdf5'), #{val_loss:.2f}
    tf.keras.callbacks.LearningRateScheduler(lambda epoch: 1e-3 * pow(0.75, floor(3 / 1)))
]

model.load("../data/weights.01.hdf5")
model.model.fit_generator(
    gen, verbose=1, use_multiprocessing=False, workers=4,
    steps_per_epoch=1000,
    callbacks=callbacks)

1000/1000 [==============================] - 292s 292ms/step - loss: 0.2066


In [16]:
model.model.fit_generator(
    gen, verbose=1, use_multiprocessing=False, workers=4,
    steps_per_epoch=1000,
    callbacks=callbacks)

1000/1000 [==============================] - 273s 273ms/step - loss: 0.1671


In [ ]:
#model.load("../data/weights.01.hdf5")

In [10]:
import numpy as np

def evaluate(model, df):
    labels = df['Label'].values    
    gen = util.RsnaDataGenerator(df, 64, (224, 224), img_dir=config['train_path'])
    preds = model.predict_generator(gen, verbose=1, workers=4)
    score = util.weighted_loss_metric(labels, preds[:len(labels)])
    return score



In [17]:
score = evaluate(model.model, df.iloc[valid_idx]) 
print(score)

1054/1054 [==============================] - 83s 79ms/step
0.12450761051701204
